In [111]:
import numpy as np 
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import GradientBoostingClassifier


In [2]:
df_players = pd.read_csv(r'data/players.csv')
df_week1 = pd.read_csv(r'data/week1.csv')
df_week2 = pd.read_csv(r'data/week2.csv')
df_week3 = pd.read_csv(r'data/week3.csv')
df_week4 = pd.read_csv(r'data/week4.csv')
df_week5 = pd.read_csv(r'data/week5.csv')
df_week6 = pd.read_csv(r'data/week6.csv')
df_week7 = pd.read_csv(r'data/week7.csv')
df_week8 = pd.read_csv(r'data/week8.csv')

In [3]:
columns = ["nflId","height","weight","officialPosition"]

In [4]:
column_weeks = ['gameId','playId','nflId','s','a']

In [5]:
df_week1 = df_week1[column_weeks].dropna(axis=0)
df_week2 = df_week2[column_weeks].dropna(axis=0)
df_week3 = df_week3[column_weeks].dropna(axis=0)
df_week4 = df_week4[column_weeks].dropna(axis=0)
df_week5 = df_week5[column_weeks].dropna(axis=0)
df_week6 = df_week6[column_weeks].dropna(axis=0)
df_week7 = df_week7[column_weeks].dropna(axis=0)
df_week8 = df_week8[column_weeks].dropna(axis=0)
weeks_list = [df_week1,df_week2,df_week3,df_week4,df_week5,df_week6,df_week7,df_week8]

In [6]:
joined_weeks_list = pd.concat(weeks_list,axis=0)

In [7]:
mean_speed = joined_weeks_list.groupby(['nflId']).mean().reset_index()[['nflId','s']]
mean_acceleration = joined_weeks_list.groupby(['nflId']).mean().reset_index()[['nflId','a']]


In [8]:
df1 = df_players.merge(mean_acceleration,on='nflId')
df = df1.merge(mean_speed,left_on='nflId',right_on='nflId')
df = df.rename(columns={'a':'a_avg','s':'s_avg'})

In [9]:
df = df[['height','weight','officialPosition','a_avg','s_avg']]
for i in range(len(df)):
    df['height'][i] = int(df['height'][i][0])*0.3048 + int(df['height'][i][2])*0.3048/10

C:\Users\User\AppData\Local\Temp\ipykernel_10416\2211777729.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['height'][i] = int(df['height'][i][0])*0.3048 + int(df['height'][i][2])*0.3048/10


In [61]:
y = df['officialPosition']
x = df.drop('officialPosition',axis=1)
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2)

In [ ]:
#Support Vector machine

In [80]:
clf = svm.SVC(C= 100)
clf.fit(x_train,y_train)

SVC(C=100)

In [79]:
y_pred = clf.predict(x_test)
# print("Accuracy:", accuracy_score(y_test,y_pred))
print(clf.score(x_test,y_test))


0.3898809523809524


In [ ]:
parameters = {
    'C':np.logspace(-2,1,4),
    'kernel':['poly','linear','rbf','sigmoid']    
}

clf_1 = svm.SVC()


In [ ]:
gs = GridSearchCV(estimator = clf_1, param_grid=parameters)

In [ ]:
gs.fit(x_train,y_train)

In [ ]:
print(gs.best_estimator_)
print(gs.best_score_)
print(gs.best_params_)

In [ ]:
#Naive Bayes algorithim

In [60]:
x_train_b, x_test_b, y_train_b, y_test_b = train_test_split(x, y, test_size=0.5, random_state=42)


In [15]:
mnb = MultinomialNB()
mnb.fit(x_train_b, y_train_b)
y_pred = mnb.predict(x_test_b)

In [86]:
mnb.score(x_test_b,y_test_b)

0.32976190476190476

In [ ]:
#Logistic Regression

In [17]:
x_train_lg , x_test_lg, y_train_lg, y_test_lg = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
parameters = {
    'C':np.logspace(0,1,50),
    'penalty':['l2'],
    'solver':['newton-cg','newton-cholesky','sag','liblinear']
    
    
}

In [18]:
lr = LogisticRegression()
#params found from the gridsearch below
lr.fit(x_train_lg,y_train_lg)


C:\Users\User\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [20]:
LogisticRegression(C=7, max_iter=1000, penalty='l1', solver='liblinear')
lr.score(x_test_lg,y_test_lg)

0.44642857142857145

In [ ]:
gs = GridSearchCV(estimator = lr,param_grid=parameters)

In [ ]:
gs.fit(x_train_lg,y_train_lg)

In [ ]:
print(gs.best_estimitor_)

In [67]:
lr_best = LogisticRegression(penalty='l1',solver='liblinear', C= 7,max_iter = 1000)
lr_best.fit(x_train,y_train)
lr_best.score(x_test,y_test)

0.6398809523809523

In [ ]:
#random forest classifier

In [ ]:
rf = RandomForestClassifier()
params_rf = {
    'n_estimators': [50, 100, 200,300]
}
rf_gs = GridSearchCV(rf, params_rf, cv=5)
rf_gs.fit(x_train, y_train)

In [ ]:
print(rf_gs.score(x_test,y_test))
print(rf_gs.best_params_)
print(rf_gs.best_score_)

In [66]:
rf_best = RandomForestClassifier(n_estimators=100,)
rf_best.fit(x_train,y_train)
rf_best.score(x_test,y_test)

0.7083333333333334

In [ ]:
"""
Ensemble methods: 

Naive bayes and logistic regression are both weak models so i am going to apply boosting algorithims to them both 
then i will apply an ensemble bagging method with the base_estimator as the default decision tree classifier. After that 
i will stack all of these models together. 


"""


In [110]:
#Boosting ]

0.11094564408041697

In [ ]:
bayes_ada_1 = AdaBoostClassifier(base_estimator = mnb,algorithm='SAMME.R')
bayes_ada_1.fit(x_train,y_train)
bayes_ada_1.score(x_train,y_train)

In [113]:
gbc = GradientBoostingClassifier()
gbc.fit(x_train,y_train)
gbc.score(x_test,y_test)

0.7083333333333334

In [108]:

bag_class_1 = BaggingClassifier(mnb)
bag_class_1.fit(x_train,y_train)
bag_class_1.score(x_test,y_test)

0.37202380952380953

0.11904761904761904

In [38]:
#Bagging

In [78]:
bag_class = BaggingClassifier()
bag_class.fit(x_train,y_train)
bag_class.score(x_test,y_test)

0.6815476190476191

In [ ]:
#Stacking

In [114]:
estimators  = [('bg',bag_class),('rf',rf_best),('lr',lr_best),('gbc',gbc)]

In [115]:
stacked = StackingClassifier(estimators)
x_train_final ,x_test_final,y_train_final,y_test_final = train_test_split(x,y,test_size = 0.3)

In [116]:
stacked.fit(x_train_final,y_train_final)

C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:1092: RuntimeWarning: Number of classes in training fold (18) does not match total number of classes (19). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:676: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\User\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:1092: RuntimeWarning: Number of classes in training fold (18) does not match total number of classes (19). Results may not be appropriate for your use case. To fix this, use a cross-validation 

StackingClassifier(estimators=[('bg', BaggingClassifier()),
                               ('rf', RandomForestClassifier()),
                               ('lr',
                                LogisticRegression(C=7, max_iter=1000,
                                                   penalty='l1',
                                                   solver='liblinear')),
                               ('gbc', GradientBoostingClassifier())])

In [117]:
stacked.score(x_test_final,y_test_final)
#stacked ensemble method across several models is less accurate than inidividual models such as 
#gradientboosting and randomforest

0.6805555555555556